In [1]:
import pandas as pd
import numpy as np
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode

In [2]:
df1=pd.read_excel('./优惠占比上升分析.xlsx',sheet_name='8月')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412629 entries, 0 to 412628
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   交易日期     412629 non-null  int64  
 1   商户号      412629 non-null  int64  
 2   商户名称     412629 non-null  object 
 3   所属行业     412629 non-null  object 
 4   入网日期     412629 non-null  int64  
 5   商户类型     412629 non-null  object 
 6   一级代理商编号  412629 non-null  int64  
 7   一级代理商名称  412629 non-null  object 
 8   补贴类型     412629 non-null  object 
 9   交易笔数     412629 non-null  int64  
 10  交易金额     412629 non-null  float64
 11  手续费      412629 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 37.8+ MB


In [3]:
df2=pd.read_excel('./优惠占比上升分析.xlsx',sheet_name='9月')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442307 entries, 0 to 442306
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   交易日期     442307 non-null  int64  
 1   商户号      442307 non-null  int64  
 2   商户名称     442307 non-null  object 
 3   所属行业     442307 non-null  object 
 4   入网日期     442307 non-null  int64  
 5   商户类型     442307 non-null  object 
 6   一级代理商编号  442307 non-null  int64  
 7   一级代理商名称  442307 non-null  object 
 8   补贴类型     442307 non-null  object 
 9   交易笔数     442307 non-null  int64  
 10  交易金额     442307 non-null  float64
 11  手续费      442307 non-null  float64
dtypes: float64(2), int64(5), object(5)
memory usage: 40.5+ MB


In [4]:
# 修改一下数据格式
columns=['交易日期','商户号','入网日期','一级代理商编号']
for i in columns:
    for j in [df1,df2]:
            j[i]=j[i].map(str)    

In [6]:
# 定义一个函数返回某个维度下的优惠占比
def yhzb(df,col):
    jyje=df.groupby(col).sum()['交易金额'].reset_index()
    yhje=df[df['补贴类型']=='公司补贴'].groupby(col).sum()['交易金额'].reset_index()
    yh=pd.merge(jyje,yhje,on=col)
    yh.columns=[col,'交易金额','优惠金额']
    yh['优惠占比']=yh['优惠金额']/yh['交易金额']
#     yh['优惠占比']=yh['优惠占比'].apply(lambda x:'%.2f%%'%(x*100))
    return yh

In [7]:
# 先看一下8月和9月不同日期的优惠占比并作图
mryh1=yhzb(df1,'交易日期')
mryh2=yhzb(df2,'交易日期')
mryh1

,交易日期,交易金额,优惠金额,优惠占比
0,20210824,9.584631e+07,29686821.42,0.309734
1,20210825,1.005292e+08,31042889.58,0.308795
2,20210826,1.005392e+08,29780573.71,0.296209
3,20210827,1.044962e+08,30730237.07,0.294080
4,20210828,1.065984e+08,30753225.23,0.288496


In [8]:
mryh2

,交易日期,交易金额,优惠金额,优惠占比
0,20210914,8.805468e+07,31734899.65,0.360400
1,20210915,8.117194e+07,29277683.04,0.360687
2,20210916,8.061498e+07,28350249.21,0.351675
3,20210917,7.827491e+07,28021259.59,0.357985
4,20210918,8.150158e+07,29444676.87,0.361277


In [9]:
mryh=pd.concat([mryh1,mryh2],axis=0)

data_x = mryh['交易日期'].tolist()
data_y =  mryh['优惠占比'].tolist()
data_y_0 = [round(x*100,2) for x in data_y]

line = (
    Line()
        .add_xaxis(data_x)
        .add_yaxis("优惠占比", data_y_0, label_opts=opts.LabelOpts(formatter=JsCode("function (params) {return params.value[1] + '%'}")))
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %")))
)
line.render_notebook()

从图中可以看出9.14-9.18这段时间的优惠占比比8.24-8.28这段时间的优惠占比高了很多，接下来看下是什么原因导致的

In [10]:
# 先看下从行业维度来看下是不是有问题
df1['所属行业']=df1['所属行业'].apply(lambda x:x.strip())
df2['所属行业']=df2['所属行业'].apply(lambda x:x.strip())
# 行业字段有很多空格所以需要先去除空格一下

In [11]:
df1_hyyh=yhzb(df1,'所属行业')
df2_hyyh=yhzb(df2,'所属行业')
hyyh_db=pd.merge(df1_hyyh,df2_hyyh,on='所属行业',how='right')
hyyh_db.head()

,所属行业,交易金额_x,优惠金额_x,优惠占比_x,交易金额_y,优惠金额_y,优惠占比_y
0,个人咨询服务,1258363.08,22405.85,0.017806,1104597.08,27090.40,0.024525
1,中医诊所,25741.00,8588.00,0.333631,14802.00,10879.00,0.734968
2,乐器商店,167364.00,4564.00,0.027270,74198.50,2126.50,0.028660
3,书店,290582.39,60148.45,0.206993,377178.20,183057.24,0.485334
4,书报批发,3427.08,534.08,0.155841,6.00,6.00,1.000000


In [12]:
hyyh_db['上升百分点']=hyyh_db['优惠占比_y']-hyyh_db['优惠占比_x']
hyyh_db.sort_values(['交易金额_y'],inplace=True,ascending=False)
hyyh_db['上升百分点']=hyyh_db['上升百分点'].apply(lambda x:'%.2f%%'%(x*100))
hyyh_db['9月累计优惠金额占比']=(hyyh_db['交易金额_y']/(hyyh_db['交易金额_y'].sum())).cumsum()
hyyh_db['9月累计优惠金额占比']=hyyh_db['9月累计优惠金额占比'].apply(lambda x:'%.2f%%'%(x*100))
top10=hyyh_db[:10]
top10

,所属行业,交易金额_x,优惠金额_x,优惠占比_x,交易金额_y,优惠金额_y,优惠占比_y,上升百分点,9月累计优惠金额占比
95,就餐场所和餐馆,5.283921e+07,26801374.31,0.507225,55909209.01,35921518.21,0.642497,13.53%,13.65%
213,百货商店,7.391374e+07,19753569.63,0.267252,52308408.88,17413173.28,0.332894,6.56%,26.43%
61,各类杂货店、便利店,3.617200e+07,11565834.61,0.319746,29650247.57,11982399.19,0.404125,8.44%,33.67%
302,食品,2.906815e+07,13617417.33,0.468465,20625723.25,12149151.89,0.589029,12.06%,38.70%
30,其他,1.802571e+07,8554643.92,0.474580,16325798.76,9244930.02,0.566277,9.17%,42.69%
109,快餐店,1.276834e+07,7958880.06,0.623329,14017136.81,10641485.38,0.759177,13.58%,46.11%
233,综合性超级市场,1.706917e+07,7690617.44,0.450556,10926577.84,5298785.20,0.484945,3.44%,48.78%
17,保健、美容及洗浴服务,1.215388e+07,2966814.19,0.244104,9826198.96,1732460.75,0.176310,-6.78%,51.18%
144,服装综合零售店,1.575389e+07,3655895.77,0.232063,9191468.65,1634638.91,0.177843,-5.42%,53.42%
12,休闲饮品场所、酒吧、咖啡厅、茶馆,8.463219e+06,2015658.28,0.238167,8092682.70,2155804.88,0.266389,2.82%,55.40%


In [13]:
data_x = top10['所属行业'].tolist()
data_y =  top10['上升百分点'].tolist()
data_y_0 = [float(x.strip('%')) for x in data_y]

chart1 = (
    Bar()
        .add_xaxis(data_x)
        .add_yaxis("优惠占比上升百分点", data_y_0, label_opts=opts.LabelOpts(formatter="{c}%"))
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %")))
)


data_y2 =  top10['9月累计优惠金额占比'].tolist()
data_y2_0 = [float(x.strip('%')) for x in data_y2]
chart2 = (
        Line().add_xaxis(data_x)
        .add_yaxis("9月累计优惠金额占比", data_y2_0, label_opts=opts.LabelOpts(formatter="{c}%"))
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %")))
)
chart2.overlap(chart1)

chart2.set_global_opts(
    xaxis_opts=opts.AxisOpts(
        axislabel_opts=opts.LabelOpts( 
            font_size=10, 
            rotate=45 ) ) )

chart2.render_notebook()

从图中可以看出排名前十的行业的优惠金额占了总的55.4%，影响还是比较大的，其中餐饮类行业优惠占比均上升了10%以上，说明餐饮类行业优惠占比的上升对整体的优惠占比的上升产生了一定的影响

**接下来看下是不是新增商户的问题**

In [14]:
lssh=df2[~df2['商户号'].isin(df1['商户号'].values)]
jyje=lssh['交易金额'].sum()
yhje=lssh[lssh['补贴类型']=='公司补贴']['交易金额'].sum()
zb=yhje/jyje
lssh_zb=pd.DataFrame({'交易金额':jyje,'优惠金额':yhje,'优惠占比':zb},index=[0])
lssh_zb['优惠占比']=lssh_zb['优惠占比'].map(lambda x : format(x, '.2%'))
lssh_zb

,交易金额,优惠金额,优惠占比
0,80811581.76,24926070.23,30.84%


In [15]:
xzsh=df2[~df2['商户号'].isin(lssh['商户号'].values)]
jyje=xzsh['交易金额'].sum()
yhje=xzsh[xzsh['补贴类型']=='公司补贴']['交易金额'].sum()
zb=yhje/jyje
xzsh_zb=pd.DataFrame({'交易金额':jyje,'优惠金额':yhje,'优惠占比':zb},index=[0])
xzsh_zb['优惠占比']=xzsh_zb['优惠占比'].map(lambda x : format(x, '.2%'))
xzsh_zb

,交易金额,优惠金额,优惠占比
0,3.288065e+08,1.219027e+08,37.07%


In [19]:
shzb=pd.DataFrame({'新增商户优惠占比':xzsh_zb['优惠占比'].values,'流失商户优惠占比':lssh_zb['优惠占比'].values},index=[0])
data_x = shzb.columns.tolist()
data_y =  shzb.loc[0].tolist()
data_y_0 = [float(x.strip('%')) for x in data_y]

bar = (
    Bar()
        .add_xaxis(data_x)
        .add_yaxis("优惠占比", data_y_0, label_opts=opts.LabelOpts(formatter="{c}%"))
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %")))
)

bar.render_notebook()

从图中可以看出流失商户的优惠占比比新增商户的优惠占比低了7%左右，所以优质客户的流失也是对优惠占比的上升产生了一定影响

**接下来看下商户类型是否对优惠占比上升产生了影响**

In [20]:
shlx1=yhzb(df1,'商户类型')
shlx2=yhzb(df2,'商户类型')
shlx1['交易占比']=(shlx1['交易金额']/(shlx1['交易金额'].sum())).map(lambda x : format(x, '.2%'))
shlx1['优惠金额占比']=(shlx1['优惠金额']/(shlx1['优惠金额'].sum())).map(lambda x : format(x, '.2%'))
shlx1

,商户类型,交易金额,优惠金额,优惠占比,交易占比,优惠金额占比
0,小微商户,9.692271e+07,3.290255e+07,0.339472,19.08%,21.65%
1,普通商户,4.110866e+08,1.190912e+08,0.289699,80.92%,78.35%


In [21]:
shlx2['交易占比']=(shlx2['交易金额']/(shlx2['交易金额'].sum())).map(lambda x : format(x, '.2%'))
shlx2['优惠金额占比']=(shlx2['优惠金额']/(shlx2['优惠金额'].sum())).map(lambda x : format(x, '.2%'))
shlx2

,商户类型,交易金额,优惠金额,优惠占比,交易占比,优惠金额占比
0,小微商户,7.110684e+07,2.912355e+07,0.409575,17.36%,19.84%
1,普通商户,3.385113e+08,1.177052e+08,0.347714,82.64%,80.16%


对比2个时间段的不同商户类型的优惠占比发现，优惠占比均有所提升，提升幅度都在6%左右，所以商户类型的构成对优惠占比上升并无影响

In [22]:
rw1=yhzb(df1,'入网日期')
rw2=yhzb(df2,'入网日期')
rwdb=pd.merge(rw1,rw2,on='入网日期',how='right')
rwdb.dropna(inplace=True)
rwdb['上升百分点']=rwdb['优惠占比_y']-rwdb['优惠占比_x']
rwdb.sort_values('交易金额_y',ascending=False,inplace=True)
rwdb['上升百分点']=rwdb['上升百分点'].map(lambda x : format(x, '.2%'))
rwdb[:10]

,入网日期,交易金额_x,优惠金额_x,优惠占比_x,交易金额_y,优惠金额_y,优惠占比_y,上升百分点
85,20211216,8366054.83,2390080.76,0.285688,6961913.27,2119012.71,0.304372,1.87%
77,20211208,8401958.30,2695974.42,0.320875,6727789.14,2332110.71,0.346638,2.58%
78,20211209,9174375.02,2360022.31,0.257241,6620735.06,2375227.05,0.358756,10.15%
83,20211214,8414050.35,2276388.21,0.270546,6236142.59,2238171.49,0.358903,8.84%
84,20211215,7800005.80,2432795.41,0.311897,6109376.21,2202582.46,0.360525,4.86%
114,20220114,4515360.61,1696997.44,0.375828,5821761.82,1326596.71,0.227869,-14.80%
76,20211207,6814673.88,2472260.11,0.362785,5816628.80,2280970.22,0.392146,2.94%
86,20211217,7760562.20,2362746.63,0.304456,5674874.71,2113311.34,0.372398,6.79%
79,20211210,7664356.67,2625769.25,0.342595,5612936.95,2378266.79,0.423712,8.11%
82,20211213,7118727.33,2075043.26,0.291491,5564659.26,1960602.88,0.352331,6.08%


不同入网日期商户的优惠占比上升百分点并无特别异常值，可以认为不同入网日期商户对优惠占比上升并无影响